In [ ]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

### medina from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211101_medina' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 'Sheet1'
header_row = 0
df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
# df = pd.read_csv(f'{base_dir}/{file_name}')
df = cb_utils.df_format_columns(df)
df = cb_utils.df_add_inserted_at_col(df)
df = cb_utils.df_col_format_icd10(df, 'diag')
df.head()

In [ ]:
eng = cb_utils.get_engine(source='msh_analytics')
table = file_name.replace('.xlsx', '').replace('.csv', '')

df.to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='replace', method='multi', index=True, index_label='id')

### hometown from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211101_hometown' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 'Sheet1'
header_row = 0
df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
# df = pd.read_csv(f'{base_dir}/{file_name}')
df = cb_utils.df_format_columns(df)
df = cb_utils.df_add_inserted_at_col(df)
df = cb_utils.df_col_format_icd10(df, 'diag')
df.head()

In [ ]:
eng = cb_utils.get_engine(source='msh_analytics')
table = file_name.replace('.xlsx', '').replace('.csv', '')

df.to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='replace', method='multi', index=True, index_label='id')

### BCBS for eastwood from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211029_bcbs_from_bennett' in f]
file_names

In [ ]:
file_name = file_names[0]
# df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df = pd.read_csv(f'{base_dir}/{file_name}')
df = cb_utils.df_format_columns(df)
df = cb_utils.df_add_inserted_at_col(df)
df = cb_utils.df_col_format_icd10(df, 'diag')
df.head()

In [ ]:
eng = cb_utils.get_engine(source='msh_analytics')
table = file_name.replace('.xlsx', '').replace('.csv', '')

df.to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='replace', method='multi', index=True, index_label='id')

### Recap for eastwood from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211027' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 're-capture report 20211027'
header_row = 2
# df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df = pd.read_csv(f'{base_dir}/{file_name}')
# df.head()

In [ ]:
df = cb_utils.df_col_format_icd10(df, 'Diagnosis Code')
df.head()

In [ ]:
df.columns = [re.sub('\(|\)| |\/', '_', c).lower() for c in df.columns]
df.head()

In [ ]:

table = file_name.replace('.xlsx', '').replace('.csv', '')

In [ ]:
df.to_sql(f'external_emr_data_eastwood_recap_20211027', eng, schema='junk', if_exists='replace', method='multi', index=False)

### Recap for eastwood from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211027' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 're-capture report 20211027'
header_row = 2
# df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df = pd.read_csv(f'{base_dir}/{file_name}')
# df.head()

In [ ]:
df = cb_utils.df_col_format_icd10(df, 'Diagnosis Code')
df.head()

In [ ]:
df.columns = [re.sub('\(|\)| |\/', '_', c).lower() for c in df.columns]
df.head()

In [ ]:
df.to_sql(f'external_emr_data_eastwood_recap_20211027', eng, schema='junk', if_exists='replace', method='multi', index=False)

### Premier data from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211022_bennett_premier' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 'ICD report'
header_row = 0
# df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df = pd.read_csv(f'{base_dir}/{file_name}', header=header_row)
df.head()

In [ ]:
# remove null icds
# df.loc[]
# df.head()
df.shape
df = df.assign(icd=df['Diagnosis Name + Code'].str.split(': ').apply(lambda x: x[0].strip().lower().replace('.','')))
df.head()

In [ ]:
df = df.assign(first_name=df.Patient.str.split(', ').apply(lambda x: x[1]), last_name=df.Patient.str.split(', ').apply(lambda x: x[0]))
df.head()

In [ ]:
table = file_name.replace('.xlsx', '').replace('.csv', '')
eng = cb_utils.get_engine(source='msh_analytics')

cols = ['Patient', 'Patient Age', 'None', 'None.1', 'Rank', 'Diagnosis Name + Code', 'Visit Count', 'icd', 'first_name', 'last_name']

df[cols].to_sql(f'external_emr_data_{table}', eng, schema='junk', if_exists='replace', method='multi', index=False)
# df[cols].to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='append', method='multi', index=False)

### Eastwood data from Bennett

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if 'bennett' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 'ICD report'
header_row = 1
df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df.head()

In [ ]:
# remove null icds
df = df.loc[~df.icd10encounterdiagcode.isna()]
df.head()

In [ ]:
df = cb_utils.df_col_format_icd10(df, 'icd10encounterdiagcode')
df.head(10)

In [ ]:
table = file_name.replace('.xlsx', '')
eng = cb_utils.get_engine(source='msh_analytics')
# df.to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='append', method='multi', index=False)

### CFM from Diane 10-21-2021

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20210727_CFM' in f]
file_names

In [ ]:
file_name = file_names[0]
sheet_name = 'Sheet1'
header_row = 0
df = pd.read_excel(f'{base_dir}/{file_name}', sheet_name=sheet_name, header=header_row)
df.head()

In [ ]:
# remove null icds
df = df.loc[~df.Active_Diagnosis.isna()]

In [ ]:
df[['Last Visit Date', 'First Visit Date', 'DOB']] = df[['Last Visit Date', 'First Visit Date', 'DOB']].astype(np.Datetime64)

In [ ]:
# split icd string
df.Active_Diagnosis = df.Active_Diagnosis.str.split(';').apply(lambda x: list(set(e.lower().strip().replace('.', '') for e in x if e != "")))

In [ ]:
for i in range(1, 16):
    df = df.assign(**{f'diag_{i}': df.Active_Diagnosis.apply(lambda x: x[i - 1] if len(x) >= i else None)})
#     df = cb_utils.df_col_format_icd10(df, f'diag_{i}')

In [ ]:
df.head()

In [ ]:
table = file_name.replace('.xlsx', '')
eng = cb_utils.get_engine(source='msh_analytics')
df.to_sql(f'external_emr_data_20210727_cfm_from_diane', eng, schema='raw', if_exists='append', method='multi', index=False)

### Hometown from Diane 10-21-2021

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211015_hometown_from_diane' in f]
file_names

In [ ]:
file_name = file_names[0]
df = pd.read_csv(f'{base_dir}/{file_name}')
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df[['Patient - DOB - Full', 'Procedure - DOS']] = df[['Patient - DOB - Full', 'Procedure - DOS']].astype(np.Datetime64)

In [ ]:
dx_cols = ['Diagnosis - Diagnosis 1',
       'Diagnosis - Diagnosis 2', 'Diagnosis - Diagnosis 3',
       'Diagnosis - Diagnosis 4', 'Diagnosis - Diagnosis 5',
       'Diagnosis - Diagnosis 6', 'Diagnosis - Diagnosis 7',
       'Diagnosis - Diagnosis 8', 'Diagnosis - Diagnosis 9',
       'Diagnosis - Diagnosis 10', 'Diagnosis - Diagnosis 11',
       'Diagnosis - Diagnosis 12']

for c in dx_cols:
    df = cb_utils.df_col_format_icd10(df, c)

In [ ]:
table = file_name.replace('.xlsx', '').replace('.csv', '')
eng = cb_utils.get_engine(source='msh_analytics')
df.to_sql(f'external_emr_data_{table}', eng, schema='raw', if_exists='append', method='multi', index=False)
# df.to_sql(f'external_emr_data_{table}', eng, schema='junk', if_exists='replace', method='multi', index=False)

# Imort External EMR data

In [ ]:
base_dir = '/Users/bp/msh/external_emr_data/'
file_names = [f for f in os.listdir(base_dir) if '20211029_all' in f and '~$' not in f]
file_names

In [ ]:
for file_name in file_names:
    emr_data = pd.read_excel(f'{base_dir}/{file_name}', sheet_name='Sheet1', header=0)
    emr_data = emr_data.rename(columns={"notes ": "notes"})
    emr_data = emr_data[['golgi_id', 'patient_name_last', 'patient_name_first', 'dob', 'location', 'captured_icd_10', 'date_captured', 'recapture_icd_10', 'date_last_captured', 'suspect_icd_10', 'notes']]
    icd_cols = ['captured_icd_10', 'recapture_icd_10', 'suspect_icd_10']
    for col in icd_cols:
        emr_data = cb_utils.df_col_format_icd10(emr_data, col)
    
    table = file_name.replace('.xlsx', '')
    eng = cb_utils.get_engine(source='msh_analytics')
    emr_data.to_sql(f'external_emr_data_{table}', eng, schema='junk', if_exists='replace', method='multi', index=False)

In [ ]:
emr_data.head(100)

In [ ]:
file_name = '20211006_medina.xlsx'
emr_data = pd.read_excel(f'{base_dir}/{file_name}', sheet_name='Sheet1', header=0)

In [ ]:
emr_data.head()

In [ ]:
name_mapping = {
    'Patient Name': 'patient_name',
    'Golgi ID': 'golgi_id', 
    'IT Upload Date': 'it_upload_date',
    'DOB': 'dob',
    'Location': 'location',
    'Insurance': 'insurance',
    'Billed ICD10s 2021': 'billed_icd10_2021',
    'DOS': 'dos',
    'Suspected ICD10': 'suspected_icd10',
    'DOS of Suspected': 'date_of_suspected_icd10',
    'Historical ICD10': 'historical_icd10',
#     'Historical ICD10-before 2021': 'historical_icd10',
}
emr_data = emr_data.rename(columns=name_mapping)
# emr_data.rename(columns=name_mapping)
emr_data.head()

In [ ]:
def format_icd10(df, col):
    df[col] = df[col].str.replace('.', '', regex=False).str.lower().str.strip()
    return df

In [ ]:
icd_cols = ['suspected_icd10', 'billed_icd10_2021', 'historical_icd10', 'missed_icd10']
for col in icd_cols:
    emr_data = format_icd10(emr_data, col)

In [ ]:
emr_data.head()
list(emr_data.columns)

In [ ]:
cols = [
 'patient_name',
 'golgi_id',
 'location',
 'insurance',
 'billed_icd10_2021',
 'dos',
 'suspected_icd10',
#  'date_of_suspected_icd10',
 'historical_icd10',
 'missed_icd10',
 'dos_of_missed',
]

In [ ]:
eng = cb_utils.get_engine(source='msh_analytics')
emr_data[cols].to_sql('external_emr_data', eng, schema='raw', if_exists='append', method='multi', index=False)

# Import Labs

In [ ]:
def rename_lab_cols(labs):
    name_mapping = {
       "creatinine_eGFR_nonafricn_am": "creatinine_egfr_nonafricn_am",
       "creatinine_eGFR_africn_am": "creatinine_egfr_africn_am",
       "hemoglobin_A1c": "hemoglobin_a1c"
    }
    return labs.rename(columns=name_mapping)

In [ ]:
base_dir = '/Users/bp/msh/labs/'
file_names = [f for f in os.listdir(base_dir) if '20211029' in f and '~$' not in f]
file_names

In [ ]:
for file_name in file_names:
    lab_data = pd.read_excel(f'{base_dir}/{file_name}', sheet_name='2021Oct29', header=0)
    lab_data = rename_lab_cols(lab_data)
    lab_data = lab_data.replace({'hiv_ag_ab_with_reflex': {'Non Reactive': 0, 'non reactive': 0, 'non reactive ': 0, 'Non Reactive ': 0, 'Reactive': 1}})
    lab_data = pd.melt(lab_data, id_vars=['patient_id', 'patient_first_name', 'patient_last_name', 'date_of_birth', 'date_of_lab_collection'], var_name='lab_type', value_name='lab_value', ignore_index=True)

    table = file_name.replace('.xlsx', '')
    eng = cb_utils.get_engine(source='msh_analytics')
    lab_data.to_sql(f'hcc_suspecting_lab_data', eng, schema='raw', if_exists='append', method='multi', index=False)
    

    # old
#     lab_data.to_sql(f'hcc_suspecting_lab_data', eng, schema='junk', if_exists='replace', method='multi', index=False)

In [ ]:
lab_data.dtypes

In [ ]:
lab_data.head(100)

In [ ]:
lab_data.hiv_ag_ab_with_reflex.value_counts()

# Old

In [ ]:
base_dir = '/Users/bp/msh/suspecting_analysis'
ndcs = pd.read_excel(f'{base_dir}/Xwalk_Master_Final_NDC.xlsx', sheet_name='RB NDC updates', header=1)

In [ ]:
ndcs.head()

In [ ]:
hcc_col = ndcs.columns[0]

ndcs = ndcs.loc[~ndcs[hcc_col].isna()]

In [ ]:
ndcs['ICD10s'] = ndcs['ICD10s'].str.split('\n|,')
ndcs = ndcs.explode('ICD10s').query("ICD10s != ''")
ndcs['ICD10s'] = ndcs['ICD10s'].str.replace('.', '', regex=False).str.lower().str.strip()

In [ ]:
ndcs.columns =  ['hcc', 'hcc_descr', 'coefficient', 'icd10', 'med', 'ndc', 'non_hcc_codes']
ndcs['med'] = ndcs['med'].str.lower().str.strip()
ndcs = ndcs.assign(ndc_code=ndcs['ndc'].str.replace('-', '', regex=False))

In [ ]:
ndcs.head()

In [ ]:
ndcs.shape

In [ ]:
eng = cb_utils.get_engine()

In [ ]:
ndcs.to_sql('ndc_xwalk', eng, schema='junk', if_exists='replace', index=False)

In [ ]:
base_dir = '/Users/bp/msh/suspecting_analysis'
ndcs = pd.read_excel(f'{base_dir}/Xwalk_Master_Final_NDC_cleanedup.xlsx', sheet_name='verified_ndc', header=0, converters={'ndc': lambda x: str(x)})

In [ ]:
ndcs.head()

In [ ]:
ndcs.to_sql('ndc_xwalk_clean', eng, schema='junk', if_exists='replace', index=False)

In [ ]:
query = f"SELECT dtw.* FROM cb.ds_tall_wide dtw JOIN cb.mcos m ON m.id = {mco_id} AND m.ds_batch_id = dtw.ds_batch_id;"
ds_tall_wide = cb_utils.sql_query_to_df(query, use_cache=use_cache)
ds_tall_wide = ds_tall_wide.drop(columns=['created_at'])
ds_tall_wide.transplant_ddos = [0 if r is None else r for r in ds_tall_wide.transplant_ddos]

In [ ]:
query = f"SELECT v.* FROM cb.ds_vaps v JOIN cb.mcos m on m.id = {mco_id} and m.ds_batch_id = v.ds_batch_id;"
# query = "SELECT * FROM cb.ds_vaps v WHERE v.ds_batch_id = 18;"
vap = cb_utils.sql_query_to_df(query, use_cache=use_cache)

sql 2021 11 01 - imports from bennet bcbs, medina, hometown
```sql
WITH
    new_dirty AS (
        INSERT
            INTO
                cb.dirty_claims(raw_id, source, is_rx, member_id, date_from, diag_code_01)
                SELECT DISTINCT
                    r.id
--                   , 'raw.external_emr_data_20211029_bcbs_from_bennett'
--                               , 'raw.external_emr_data_20211101_medina_from_bennett'
                              , 'raw.external_emr_data_20211101_hometown'
                  , FALSE
                  , m.id
                  , last_dos::date
                  , diag
                FROM
--                     raw.external_emr_data_20211029_bcbs_from_bennett r
--                     raw.external_emr_data_20211101_medina_from_bennett r
                    raw.external_emr_data_20211101_hometown r
                    JOIN cb.members m ON LOWER(m.first_name) = LOWER(r.first_name)
                        AND LOWER(m.last_name) = LOWER(r.last_name)
                        AND m.date_of_birth = r.dob::DATE
                        AND diag != 'nan'
                RETURNING id, diag_code_01
    )
INSERT
INTO
    cb.dirty_claims_diagnosis (dirty_claim_id, diag_sequence, diag, icd10_id)
SELECT
    nd.id
  , 1
  , nd.diag_code_01
  , i.id
FROM
    new_dirty nd
    JOIN ref.icd10s i ON i.icd10 = nd.diag_code_01;
```

sql 20211022

begin;
ROLLBACK ;
WITH
    dc AS (
        INSERT INTO cb.dirty_claims(raw_id, source, is_rx, member_id, date_from, diag_code_01)
            SELECT
                r.id
              , 'raw.external_emr_data_20211022_bennett_premier'
              , FALSE
              , m.id
              , '2020-12-31'
              , icd
            FROM
                raw.external_emr_data_20211022_bennett_premier r
                JOIN members m ON
                         LOWER(m.first_name) = REGEXP_REPLACE(LOWER(r.first_name), ' [a-z]$', '')
                    AND LOWER(m.last_name) = LOWER(r.last_name)
                    AND m.date_of_birth BETWEEN r.dob_start AND r.dob_end
            RETURNING id, diag_code_01
    )
INSERT
INTO
    dirty_claims_diagnosis (dirty_claim_id, diag_sequence, diag, icd10_id)
SELECT dc.id, 1, diag_code_01, i.id
FROM
    dc
join ref.icd10s i on diag_code_01 = i.icd10
;

SQl importing emr exports 20211021


WITH
    dc AS
        (
            INSERT
                INTO
                    dirty_claims(raw_id, source, is_rx, member_id, date_from, date_to, diag_code_01, inserted_at,
                                 updated_at)
                    SELECT
                        j.id
                      , 'raw.external_emr_data_20201019_eastwood_bennett'
                      , FALSE
                      , m.id
                      , "cln enc date"
                      , "cln enc date"
                      , icd10encounterdiagcode
                      , NOW()
                      , NOW()
                    FROM
                        raw.external_emr_data_20201019_eastwood_bennett j
                        JOIN members m ON LOWER(m.first_name) = LOWER(j."patient firstname")
                            AND LOWER(m.last_name) = LOWER(j."patient lastname")
                            AND m.date_of_birth = j.patientdob::DATE
                    RETURNING id, diag_code_01
        )
INSERT
INTO
    dirty_claims_diagnosis (dirty_claim_id, diag_sequence, diag, icd10_id)
SELECT
    dc.id
  , 1
  , dc.diag_code_01
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_01
;

SELECT *
FROM
    junk."external_emr_data_20210727_CFM";
 ;
SELECT
FROM
    raw.external_emr_data_20210727_cfm_from_diane ;



WITH
    dc AS
        (
            INSERT
                INTO
                    dirty_claims(raw_id, source, is_rx, member_id, date_from, date_to, diag_code_01, inserted_at,
                                 updated_at,  diag_code_02
  , diag_code_03
  , diag_code_04
  , diag_code_05
  , diag_code_06
  , diag_code_07
  , diag_code_08
  , diag_code_09
  , diag_code_10
  , diag_code_11
  , diag_code_12
  , diag_code_13
  , diag_code_14
  , diag_code_15)
                    SELECT
                        j.id
                      , 'raw.external_emr_data_20210727_cfm_from_diane'
                      , FALSE
                      , m.id
                      , "Last Visit Date"
                         , null
                      , diag_1
                      , NOW()
                      , NOW()
                    , diag_2
                      , diag_3
                      , diag_4
                      , diag_5
                      , diag_6
                      , diag_7
                      , diag_8
                      , diag_9
                      , diag_10
                      , diag_11
                      , diag_12
                      , diag_13
                      , diag_14
                      , diag_15
                    FROM
                        raw.external_emr_data_20210727_cfm_from_diane j
                        JOIN members m ON LOWER(m.first_name) = LOWER(j."First Name")
                            AND LOWER(m.last_name) = LOWER(j."Last Name")
                            AND m.date_of_birth = j."DOB"::DATE
                    RETURNING id , raw_id
        )
, unnested as (
 SELECT
    dc.id
  ,  UNNEST(r."Active_Diagnosis"::TEXT[]) icd
FROM
    dc
        join raw.external_emr_data_20210727_cfm_from_diane r on r.id = dc.raw_id
)
INSERT
INTO
    dirty_claims_diagnosis (dirty_claim_id, diag_sequence, diag, icd10_id)

SELECT un.id
     , row_number() OVER (PARTITION BY un.id)
     , un.icd
    , i.id
    from unnested un
    JOIN ref.icd10s i ON i.icd10 = un.icd
;


    DROP TABLE IF EXISTS  junk."external_emr_data_20211015_hometown_from_diane.csv";


SELECT
    id
  , "Encounter - ID"
  , "Patient - ID"
  , "Patient - Name - First"
  , "Patient - Name - Last"
  , "Rendering Provider - NPI"
  , "Patient - DOB - Full"
  , "Procedure - DOS"
  , "Diagnosis - Diagnosis 1"

  , inserted_at
FROM
    raw.external_emr_data_20211015_hometown_from_diane;

-- begin;
-- ROLLBACK ;
-- commit;
SELECT *
FROM
    cb.dirty_claims dc
    JOIN dirty_claims_diagnosis dcd ON dc.id = dcd.dirty_claim_id
WHERE source =   'raw.external_emr_data_20211015_hometown_from_diane'

;

WITH
    dc AS
        (
            INSERT
                INTO
                    dirty_claims(raw_id, source, is_rx, member_id, date_from, date_to, diag_code_01, inserted_at,
                                 updated_at,  diag_code_02
  , diag_code_03
  , diag_code_04
  , diag_code_05
  , diag_code_06
  , diag_code_07
  , diag_code_08
  , diag_code_09
  , diag_code_10
  , diag_code_11
  , diag_code_12
)
                    SELECT
                        j.id
                      , 'raw.external_emr_data_20211015_hometown_from_diane'
                      , FALSE
                      , m.id
                         , "Procedure - DOS"
                      , "Procedure - DOS"
                         , "Diagnosis - Diagnosis 1"
                      , NOW()
                      , NOW()
                    , "Diagnosis - Diagnosis 2"
  , "Diagnosis - Diagnosis 3"
  , "Diagnosis - Diagnosis 4"
  , "Diagnosis - Diagnosis 5"
  , "Diagnosis - Diagnosis 6"
  , "Diagnosis - Diagnosis 7"
  , "Diagnosis - Diagnosis 8"
  , "Diagnosis - Diagnosis 9"
  , "Diagnosis - Diagnosis 10"
  , "Diagnosis - Diagnosis 11"
  , "Diagnosis - Diagnosis 12"
                    FROM
                        raw.external_emr_data_20211015_hometown_from_diane j
                        JOIN members m ON LOWER(m.first_name) = LOWER(j."Patient - Name - First")
                            AND LOWER(m.last_name) = LOWER(j."Patient - Name - Last")
                            AND m.date_of_birth = j."Patient - DOB - Full"
                    RETURNING *
        )
INSERT
INTO
    dirty_claims_diagnosis (dirty_claim_id, diag_sequence, diag, icd10_id)
SELECT
    dc.id
  , 1
  , dc.diag_code_01
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_01
union
SELECT
    dc.id
  , 2
  , dc.diag_code_02
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_02
WHERE diag_code_02 != 'nan'
union
SELECT
    dc.id
  , 3
  , dc.diag_code_03
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_03
WHERE diag_code_03 != 'nan'
union
SELECT
    dc.id
  , 4
  , dc.diag_code_04
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_04
WHERE diag_code_04 != 'nan'
union
SELECT
    dc.id
  , 5
  , dc.diag_code_05
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_05
WHERE diag_code_05 != 'nan'
union
SELECT
    dc.id
  , 6
  , dc.diag_code_06
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_06
WHERE diag_code_06 != 'nan'
union
SELECT
    dc.id
  , 7
  , dc.diag_code_07
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_07
WHERE diag_code_07 != 'nan'
union
SELECT
    dc.id
  , 8
  , dc.diag_code_08
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_08
WHERE diag_code_08 != 'nan'
union
SELECT
    dc.id
  , 9
  , dc.diag_code_09
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_09
WHERE diag_code_09 != 'nan'
union
SELECT
    dc.id
  , 10
  , dc.diag_code_10
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_10
WHERE diag_code_10 != 'nan'
union
SELECT
    dc.id
  , 11
  , dc.diag_code_11
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_11
WHERE diag_code_11 != 'nan'
union
SELECT
    dc.id
  , 12
  , dc.diag_code_12
  , i.id
FROM
    dc
    JOIN ref.icd10s i ON i.icd10 = dc.diag_code_12
WHERE diag_code_12 != 'nan'

;